The classification is performed by following:

 1. The function's energy and power are computed with definite integral with bounds from  $0$ to $10^2$, $10^3$, $10^4$, ... $10^7$ and a series is formed:
     $$
     E_1 = \int_0 ^ {10^2} f^2(x) dx \ \ \ \
     E_2 = \int_0 ^ {10^3} f^2(x) dx \ \ \ \
     E_3 = \int_0 ^ {10^4} f^2(x) dx \ \ \ \
     \cdots
     $$
     
     $$
     P_1 = \frac{1}{10^2}\int_0 ^ {10^2} f^2(x) dx \ \ \ \
     P_2 = \frac{1}{10^3}\int_0 ^ {10^3} f^2(x) dx \ \ \ \
     P_3 = \frac{1}{10^4}\int_0 ^ {10^4} f^2(x) dx \ \ \ \
     \cdots
     $$
     
     From these it is checked whether the series consists zeros or infinities and futher series is checked if it converges.
     
     In the program, integrals are evaluvated using Trapezoidal rule (https://en.wikipedia.org/wiki/Trapezoidal_rule#Uniform_grid). A wrapper function $e(x)$ is used in summation. This function checks for overflows and zero divisions and handles them appropriately:
     
     $$e(x) = 
       \begin{cases}
        \infty             &, \text{if } f^2(x)dx \text{ overflows or } f(x) \text{ involves zero division}\\
        f(x) ^ 2 dx &, \text{otherwise}
       \end{cases}
     $$
Here stepsize $dx$ is choosen to 0.1

Integration is done by:
     $$ E = \frac{e(start) + e(end)}{2} + \sum_{start+dx} ^ {end-dx} e(x) $$ 
     
$$ P = \frac{e(start) + e(end)}{2\times duration} + \sum_{start+dx} ^ {end-dx} \frac{e(x)}{duration} $$
 
Where $duration = end-start$ 

## Source code

In [1]:
from math import inf

In [2]:
def compute_E_P(f, start, end, dx):
    """
    Computes energy and power of given signal f and integrates it from
    start to finish with step size dx
    """
    duration  = end-start
    power_overflown = False
    
    def e(x):
        # wrapper function to compute integrants. It also check overlfow
        # and zero division
        try: return f(x) ** 2 * dx
        except (OverflowError, ZeroDivisionError):
            return inf
    
    # using trapezoidal rule to integerate
    
    # add initial parts
    E = (e(start) + e(end)) / 2
    P = E / duration
    
    x = start + dx
    while (
        x < end and
        P != inf # check if power has overflown, if so break the loop
    ):
        i = e(x)
        E += i
        P += i / duration
        
        x += dx
    
    return (E, P)

In [3]:
def is_converging_finite_nonzero(samples):
    """
        returns true if samples tend to approach a
        value and is not zero or infinity
    """
    
    # Check if samples contains zero or infinity
    if 0 in samples or inf in samples: return False
    
    # check if function stabilizes
    # compute absolute differences of successive elements
    diffs = [
        abs(samples[i-1] - samples[i]) for i in range(1, len(samples))
    ]
    # and their average
    avg_diff = sum(diffs) / len(diffs)
    
    # check if the difference keeps decreasing and stays below
    # some threshold (in this case average of differences)
    for i in range(1, len(diffs)):
        if (diffs[i-1] < diffs[i] and diffs[i] > avg_diff):
            return False
    
    return True

In [4]:
def classify_signal(f):
    """
    Function that does the final classification
    """
    Energies = []
    Powers = []
    
    for i in range(2, 7):
        EP = compute_E_P(f, 0, 10**i, 0.1)
        Energies.append(EP[0])
        Powers.append(EP[1])
    
    if is_converging_finite_nonzero(Energies):
        print("Energy signal, avg energy:", Energies[-1])
    elif is_converging_finite_nonzero(Powers):
        print("Power signal, avg power:", Powers[-1])
    else:
        print("Neither energy nor power signal")

## Testing against some signals

In [5]:
from math import sin, e
from inspect import getsource
def f1(x): return sin(x) * e**x
def f2(x): return 1/x
def exp(x): return e**x
def unit_step(x): return 1 if x >= 0 else 0
def linear(x): return x
def gaussian(x): return e**(-x**2)
def sigmoid(x): return 1/(1+e**(-x))
def sqrt(x): return x ** 0.5
def sine(x): return sin(x)


def print_function(f):
    """
    Prints the expression inside a given function
    """
    print(
        "f(x) =",
        getsource(f).strip().split("return ")[-1] 
    )

fx = [
    f1,
    f2,
    exp,
    unit_step,
    linear,
    sqrt,
    sine,
    gaussian,
    sigmoid
]

for f in fx:
    print_function(f)
    classify_signal(f)
    print("\n")

f(x) = sin(x) * e**x
Neither energy nor power signal


f(x) = 1/x
Neither energy nor power signal


f(x) = e**x
Neither energy nor power signal


f(x) = 1 if x >= 0 else 0
Power signal, avg power: 1.00000009975017


f(x) = x
Neither energy nor power signal


f(x) = x ** 0.5
Neither energy nor power signal


f(x) = sin(x)
Power signal, avg power: 0.5000001756971644


f(x) = e**(-x**2)
Energy signal, avg energy: 0.6266570686577502


f(x) = 1/(1+e**(-x))
Power signal, avg power: 0.9999989063946393


